In [ ]:
!pip install langchain-fireworks
import warnings
import openai
import json
from langchain_fireworks import Fireworks


# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
warnings.filterwarnings("ignore")

def get_fireworks_model(model_endpoint: str):
  client = openai.OpenAI( base_url = model_endpoint, api_key = "KnzTWAoO0cfGV2eEBd55mu4Uv2lDTEDAFqvQSS2sAxy1Lldf")
  return client

def get_chat_model(model = "accounts/fireworks/models/llama-v3p1-70b-instruct", max_tokens=500):
  final_answer_llm = Fireworks(
    api_key="KnzTWAoO0cfGV2eEBd55mu4Uv2lDTEDAFqvQSS2sAxy1Lldf",
    model=model,
    max_tokens=max_tokens,
    temperature=0
    )
  return final_answer_llm


def generate_chat_completion(client, model, messages, tools=None):
  chat_completion = client.chat.completions.create(
      model=model,
      messages=messages,
      tools=tools,
      temperature=0
  )
  return chat_completion

def get_tools_def():
  tools = [
      {
          "type": "function",
          "function": {
              "name": "make_recommendation_after_retrieving_logged",
              "description": "Provide a recommendation after retrieving the list of all items logged by the user on a specific date.",
              "parameters": {
                  "type": "object",
                  "properties": {
                      "date": {
                          "type": "string",
                          "description": "Retrieve the list of all items logged by the user on a given date. The date must be in the MM-DD-YYYY format."
                      },
                      "user_context": {
                          "type": "object",
                          "description": "Contextual information about the user, such as dietary preferences, goals, or restrictions.",
                          "properties": {
                              "dietary_preferences": {
                                  "type": "string",
                                  "description": "User's dietary preferences, e.g., vegan, vegetarian, low-carb."
                              },
                              "goals": {
                                  "type": "string",
                                  "description": "User's health or fitness goals, e.g., weight loss, muscle gain."
                              },
                              "restrictions": {
                                  "type": "string",
                                  "description": "Any dietary restrictions the user has, e.g., gluten-free, nut allergy."
                              }
                          }
                      },
                      "original_query": {
                          "type": "string",
                          "description": "The original query provided by the user, which will be considered in making the recommendation."
                      }
                  },
                  "required": ["date", "user_context", "original_query"]
              }
          }
      }
  ]
  return tools


def get_user_calories_target():
    return 2,576

def get_food_logged_by_date(user, date):
  return 100

def get_recommendation(query: str):
  TOOL_CALLING_MODEL = 'accounts/fireworks/models/firefunction-v2'
  LLAMA3_1_8B = "accounts/fireworks/models/llama-v3p1-8b-instruct"
  client = get_fireworks_model(model_endpoint = 'https://api.fireworks.ai/inference/v1')
  tools = get_tools_def()

  messages = [
      {"role": "system", "content": f"You are a helpful assistant with access to functions."
                                    "Use them if required. You have access to user_context. {dietary_preferences: vegan, goals: weight loss, restrictions: gluten-free}"},
      {"role": "user", "content": query}
  ]

  result = generate_chat_completion(client, TOOL_CALLING_MODEL, messages, tools)

  def make_recommendation_after_retrieving_logged(original_query:str, user_context: dict, date: str):
    print(f"getting log data for {date}")
    calories_consumed = get_food_logged_by_date("abc", date)
    calorie_target = get_user_calories_target()

    recommendation_prompt = f"""You are a helpful assistant acting as a nutritionist. The user has a daily calorie target of {calorie_target} and has already consumed {calories_consumed}. Based on this information, respond to the user's query: {original_query}. User the user preference. {user_context} Answer:"""
    print(f"final prompt: {recommendation_prompt}")
    print("\n\n")
    LLAMA3_1_70B= "accounts/fireworks/models/llama-v3p1-70b-instruct"
    final_answer_llm = get_chat_model(LLAMA3_1_70B)
    final_result = final_answer_llm(recommendation_prompt)
    return final_result

  function_call = result.choices[0].message.tool_calls[0].function
  tool_response = locals()[function_call.name](**json.loads(function_call.arguments))
  return tool_response





In [ ]:
ans = get_recommendation(query="recommend me on what should i eat today for lunch?")
print(ans)

getting log data for 08-11-2024
final prompt: You are a helpful assistant acting as a nutritionist. The user has a daily calorie target of (2, 576) and has already consumed 100. Based on this information, respond to the user's query: recommend me on what should i eat today for lunch. User the user preference. {'dietary_preferences': 'vegan', 'goals': 'weight loss', 'restrictions': 'gluten-free'} Answer:



 Based on your daily calorie target and the fact that you've already consumed 100 calories, I'd be happy to recommend a healthy and delicious vegan lunch option that's gluten-free and supports your weight loss goals. Considering your preferences, here's a suggestion: Grilled Vegetable Wrap (approx. 400 calories) Ingredients: 1 whole wheat gluten-free wrap 1/2 cup cooked black beans 1/2 cup roasted bell peppers 1/2 cup sliced cucumber 1/4 cup sliced avocado 1 tablespoon hummus 1 tablespoon chopped fresh cilantro Salt and pepper to taste This wrap is not only vegan and gluten-free but 

# create a diet and fitness plan

In [ ]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 31.9 MB/s eta 0:00:00


In [ ]:

def gen_user_plan(user_context):
  prompt= """
  <|begin_of_text|><|start_header_id|>system<|end_header_id|>
  You are a helpful fitness assistant. Your task is to create a comprehensive nutrition and workout plan based on the following user profile in JSON schema:

  Use the following user profile in json schema.
  {user_context}

  Nutrition Plan:

  Daily Caloric Intake: [Calculate based on goals and preferences]
  Macronutrient Distribution: [Suggested percentages for protein, carbs, and fats]
  Hydration: [Recommended daily water intake]

  Workout Plan:
  Frequency: [Number of days per week]
  Types of Exercises: [Include a mix of cardio, strength training, and flexibility exercises suitable for weight loss]
  Additional Tips: [Include any additional tips or recommendations for maximizing results]
  Ensure the plan is tailored to the individual’s specific needs and preferences.


  <|start_header_id|>user<|end_header_id|>
  Generate comprehensive nutrition, workout plan.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


  """
  plan_llm = get_chat_model("accounts/fireworks/models/llama-v3p1-70b-instruct", 2000)
  if not user_context:
    user_context = {
        "username": "",
        "email": "abcdef@gmail.com",
        "gender": "Male",
        "age": 25,
        "height": 170,
        "weight": 70,
        "fitness_goal": "Weight Loss",
        "fitness_timeline": 6,
        "fitness_range": "5% change",
        "timestamp": "2024-08-11T15:17:19.057813",
        "dietary_preferences": "vegan",
        "restrictions": "gluten-free"

    }
  final_str = prompt.format(user_context=user_context)
  plan_res = plan_llm(final_str)
  return plan_res

In [ ]:
from IPython.display import Markdown, display
user_context = {
        "username": "",
        "email": "abcdef@gmail.com",
        "gender": "Male",
        "age": 25,
        "height": 170,
        "weight": 70,
        "fitness_goal": "Weight Loss",
        "fitness_timeline": 6,
        "fitness_range": "5% change",
        "timestamp": "2024-08-11T15:17:19.057813",
        "dietary_preferences": "vegan",
        "restrictions": "gluten-free"

    }
plan_res_1 = gen_user_plan(user_context)
display(Markdown(plan_res_1))

 **Comprehensive Nutrition and Workout Plan**

**User Profile:**

*   Username: 
*   Email: abcdef@gmail.com
*   Gender: Male
*   Age: 25
*   Height: 170 cm
*   Weight: 70 kg
*   Fitness Goal: Weight Loss
*   Fitness Timeline: 6 weeks
*   Fitness Range: 5% change
*   Dietary Preferences: Vegan
*   Restrictions: Gluten-free

**Nutrition Plan:**

1.  **Daily Caloric Intake:**
    *   To achieve weight loss, a daily caloric deficit of 500 calories is recommended.
    *   Based on the user's profile, the daily caloric intake is calculated as follows:
        *   Basal Metabolic Rate (BMR): 1,987 calories (based on age, gender, weight, and height)
        *   Daily activity level: 1.55 (moderately active)
        *   Total Daily Energy Expenditure (TDEE): 3,076 calories
        *   Daily caloric intake for weight loss: 2,576 calories (TDEE - 500 calories)
2.  **Macronutrient Distribution:**
    *   Protein: 1.6 grams/kg body weight (112 grams) - 15% of daily calories
    *   Carbohydrates: 2-3 grams/kg body weight (140-210 grams) - 55% of daily calories
    *   Fat: 0.5-1 gram/kg body weight (35-70 grams) - 30% of daily calories
3.  **Hydration:**
    *   Recommended daily water intake: 3-4 liters

**Workout Plan:**

1.  **Frequency:**
    *   5 days per week, with 2 rest days
2.  **Types of Exercises:**
    *   **Cardio:**
        *   Brisk walking (30 minutes, 3 times a week)
        *   Jogging (20 minutes, 2 times a week)
        *   Swimming (20 minutes, 2 times a week)
    *   **Strength Training:**
        *   Squats (3 sets of 12 reps, 2 times a week)
        *   Lunges (3 sets of 12 reps, 2 times a week)
        *   Push-ups (3 sets of 12 reps, 2 times a week)
        *   Chest press (3 sets of 12 reps, 2 times a week)
        *   Lat pulldowns (3 sets of 12 reps, 2 times a week)
        *   Leg press (3 sets of 12 reps, 2 times a week)
    *   **Flexibility Exercises:**
        *   Yoga (30 minutes, 2 times a week)
        *   Stretching exercises (10-15 minutes, 3 times a week)
3.  **Additional Tips:**
    *   Warm up before each workout with 5-10 minutes of light cardio and stretching.
    *   Cool down after each workout with 5-10 minutes of stretching.
    *   Incorporate progressive overload by increasing the weight or resistance used in strength training exercises every two weeks.
    *   Listen to your body and rest when needed.
    *   Monitor progress through regular weight checks, body fat percentage measurements, and progress photos.

**Additional Recommendations:**

*   Consult with a healthcare professional or a registered dietitian to ensure the nutrition plan meets individual needs and preferences.
*   Incorporate mindfulness and stress-reducing techniques, such as meditation or deep breathing exercises, to support overall well-being.
*   Get enough sleep (7-9 hours) each night to aid in weight loss and muscle recovery.

By following this comprehensive nutrition and workout plan, the user can achieve their weight loss goal of 5% change in 6 weeks while maintaining a balanced and healthy lifestyle.